In [1]:
##This code will be used to re-run the datasets and code used in the ACTIVA model
##Import needed packages
import torch 
import time 
import scipy.sparse as sp
import scanpy as sc
import os
import ACTIVA

ModuleNotFoundError: No module named 'ACTIVA.Utils'

In [6]:
##1. Load pre-trained model for dataset 

#here is the path to a pre-trained ACTIVA model on 68k PBMC dataset
path_to_pretrained = "/home/hudaa/ACTIVA/Trained500Model/model_epoch_100_iter_0.pth"

model_dict = torch.load(path_to_pretrained)

activa = model_dict["Saved_Model"]

print(activa)

ModuleNotFoundError: No module named 'ACTIVA.Utils'

In [4]:
#Determine device to generate data from
#not sure about this step -- ask olga

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if str(device) == "cuda":
    print('Using GPU (CUDA)')
else:
    print('Using CPU')

Using GPU (CUDA)


In [5]:
##2. Input noise tensors to the generator s.t. it maps to the same 
## manifold as the single cell data

start = time.time()

# for reproducibility 
torch.manual_seed(0)

num_cells = 495
# look at the input size to the generator network of ACTIVA
latent_dim = 128;
z_g = torch.randn(num_cells, latent_dim).to(device)
# generate synthetic cells with ACTIVA
generated_cells = activa.decoder(z_g)

count_matrix = generated_cells.detach().cpu().numpy()

print(f"We generated {num_cells} cells in {time.time() - start} seconds (on {device})")

##3. Sparsify matrix
count_matrix_sparse = sp.csr_matrix(count_matrix)

NameError: name 'activa' is not defined

In [3]:
##4. Merge gene names with generated data

# read in the original data 
adata = sc.read_h5ad("/home/hudaa/ACTIVA/real500.h5ad")

# now make the count matrix to have the same size as the generated cells (since we will replace this in the next step)

sc.pp.subsample(adata, n_obs=num_cells, random_state=0, copy=False)

adata.X = count_matrix_sparse;

print(adata)

NameError: name 'num_cells' is not defined

In [ ]:
# delete the extra attributes that the original data had 
del adata.obs["cluster"]
del adata.obs["split"]
print(adata)

In [1]:
##5. Save cells as a Scanpy object

# check if the directory exists 
dir_name = 'ACTIVA-Generated'
if not os.path.isdir(dir_name):
    os.mkdir(dir_name) 
    print(f"Created {dir_name} directory") 

path = "./" + dir_name + f"/final-{num_cells}Generated.h5ad"
adata.write(path)
print(f"Saved the new cells to {path}")

NameError: name 'os' is not defined